# Apêndice A – Autodiff

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/fabiobento/ml-dl/Appendix_A_autodiff.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>
  <td>
    <a target="_blank" href="https://kaggle.com/kernels/welcome?src=https://github.com/fabiobento/ml-dl/Appendix_A_autodiff.ipynb"><img src="https://kaggle.com/static/images/open-in-kaggle.svg" /></a>
  </td>
</table>

Esse notebook veremos Este apêndice explica como funciona o recurso de diferenciação automática (*autodiff*) do PyTorch e como ele se compara a outras soluções.

## Introdução

Suponha que você defina uma função $f(x, y) = x^2y + y + 2$, e precise de suas derivadas parciais $\partial f/\partial x$ e $\partial f/\partial y$, tipicamente para realizar o gradiente descendente (ou algum outro algoritmo de otimização).

Suas principais opções são:
- diferenciação manual,
- aproximação por diferenças finitas,
- *autodiff* de modo direto (*forward-mode*) e
*autodiff* de modo reverso (*reverse-mode*).


O PyTorch implementa o *autodiff* de modo reverso, mas para entendê-lo completamente, é útil examinar as outras opções primeiro. Então, vamos passar por cada uma delas, começando pela diferenciação manual.

## Diferenciação Manual

A primeira abordagem para calcular derivadas é pegar um lápis e um pedaço de papel e usar seu conhecimento de cálculo para derivar a equação apropriada.

Primeiro vamos definir em código a função function $f(x,y)=x^2y + y + 2$:

In [1]:
def f(x, y):
    """
    Calcula o valor da função f(x, y) = x²y + y + 2.

    Args:
        x (float): O valor de x.
        y (float): O valor de y.

    Returns:
        float: O resultado da função f(x, y).
    """
    return x * x * y + y + 2

 Para a função $f(x, y)$ Definida acima, não é muito difícil; você só precisa usar cinco regras:

*   A derivada de uma constante é 0.
*   A derivada de $\lambda x$ é $\lambda$ (onde $\lambda$ é uma constante).
*   A derivada de $x^\lambda$ é $\lambda x^{\lambda - 1}$, então a derivada de $x^2$ é $2x$.
*   A derivada de uma soma de funções é a soma das derivadas dessas funções.
*   A derivada de $\lambda$ vezes uma função é $\lambda$ vezes a derivada dela.

Resolvendo analiticamente temos:

<a id="equation-a-1"></a>

**Equação A-1. Drivadas parciais de $f(x,y)$**

$$\dfrac{\partial f}{\partial x} = \dfrac{\partial (x^2y)}{\partial x}+\dfrac{\partial y}{\partial x}+\dfrac{\partial 2}{\partial x}= y\dfrac{\partial (x^2)}{\partial x}+0+0=2xy$$

$$\dfrac{\partial f}{\partial y} =  \dfrac{\partial (x^2y)}{\partial y}+\dfrac{\partial y}{\partial y}+\dfrac{\partial 2}{\partial y}=x^2 + 1+0 =x^2 + 1$$

Vamos definir essas derivadas parciais em Python:

In [2]:
def df(x, y):
    """
    Calcula as derivadas parciais da função f(x, y) = x²y + y + 2.

    Args:
        x (float): O valor de x.
        y (float): O valor de y.

    Returns:
        tuple: Uma tupla contendo (df/dx, df/dy), onde df/dx = 2*x*y e df/dy = x² + 1.
    """
    return 2*x*y, x*x + 1

Então, por exemplo $\dfrac{\partial f}{\partial x}(3,4) = 24$ e $\dfrac{\partial f}{\partial y}(3,4) = 10$.

In [3]:
df(3, 4)

(24, 10)

Perfeito! Também podemos encontrar as equações para as derivadas de segunda ordem (também conhecidas como Hessianas):

$\dfrac{\partial^2 f}{\partial x \partial x} = \dfrac{\partial (2xy)}{\partial x} = 2y$

$\dfrac{\partial^2 f}{\partial x \partial y} = \dfrac{\partial (2xy)}{\partial y} = 2x$

$\dfrac{\partial^2 f}{\partial y \partial x} = \dfrac{\partial (x^2 + 1)}{\partial x} = 2x$

$\dfrac{\partial^2 f}{\partial y \partial y} = \dfrac{\partial (x^2 + 1)}{\partial y} = 0$

Em x=3 e y=4, esses Hessianos são, respectivamente, 8, 6, 6, 0. Vamos usar as equações acima para calculá-los:

In [4]:
def d2f(x, y):
    """
    Calcula as derivadas parciais de segunda ordem (Hessiana) da função f(x, y) = x²y + y + 2.

    Args:
        x (float): O valor de x.
        y (float): O valor de y.

    Returns:
        tuple: Uma tupla contendo duas listas que representam a matriz Hessiana:
               [[∂²f/∂x∂x, ∂²f/∂x∂y], [∂²f/∂y∂x, ∂²f/∂y∂y]] = [[2y, 2x], [2x, 0]].
    """
    return [2*y, 2*x], [2*x, 0]

In [5]:
d2f(3, 4)

([8, 6], [6, 0])

Perfeito, mas isso requer algum trabalho matemático.

Não é muito difícil neste caso, mas para uma rede neural profunda, é praticamente impossível calcular as derivadas dessa maneira. Então, vamos ver várias maneiras de automatizar isso!

## Aproximação por Diferenças Finitas

Lembre-se de que a derivada $f^{'}(x_0,y_0)$ de uma função $f(x,y)$ em um ponto $(x_0,y_0)$ é a inclinação da função nesse ponto.

Mais precisamente, as derivadas parciais são definidas como o limite da inclinação de uma reta que passa por esse ponto $(x_0,y_0)$ e outro ponto $(x,y)$ na função, à medida que $x$ se aproxima infinitamente de $x_0$ e $y$ se aproxima infinitamente de $y_0$ (veja a [Equação A-2](equation-a-2)).

<a id="equation-a-2"></a>
**Equação A-2. Definição das derivadas parciais de uma função $f(x,y)$ no ponto $(x_0,y_0)$**

$$\dfrac{\partial f}{\partial x} = \displaystyle{\lim_{\epsilon \to 0}}\dfrac{f(x+\epsilon, y) - f(x, y)}{\epsilon}$$
$$\dfrac{\partial f}{\partial y} = \displaystyle{\lim_{\epsilon \to 0}}\dfrac{f(x, y+\epsilon) - f(x, y)}{\epsilon}$$

Portanto, se quiséssemos calcular a derivada parcial de $f(x, y)$ em relação a $x$ em $x = 3$ e $y = 4$, poderíamos calcular $f(3 + \epsilon, 4) – f(3, 4)$ e dividir o resultado por $\epsilon$, usando um valor muito pequeno para $\epsilon$.

Esse tipo de aproximação numérica da derivada é chamado de *aproximação por diferenças finitas*, e essa equação específica é chamada de [*quociente de diferença de Newton*](https://en-wikipedia-org.translate.goog/wiki/Difference_quotient?_x_tr_sl=en&_x_tr_tl=pt&_x_tr_hl=pt&_x_tr_pto=tc).

É exatamente isso que o código a seguir faz:

In [6]:
def derivative(f, x, y, x_eps, y_eps):
    """
    Calcula a derivada aproximada da função f em relação a x e y usando diferenças finitas.

    Args:
        f (function): A função para a qual calcular a derivada.
        x (float): O valor de x.
        y (float): O valor de y.
        x_eps (float): O pequeno incremento para x (epsilon para x).
        y_eps (float): O pequeno incremento para y (epsilon para y).

    Returns:
        float: A derivada aproximada.
    """
    return (f(x + x_eps, y + y_eps) - f(x, y)) / (x_eps + y_eps)

df_dx = derivative(f, 3, 4, 0.00001, 0)
df_dy = derivative(f, 3, 4, 0, 0.00001)

Infelizmente, o resultado é impreciso (e piora para funções mais complicadas). Os resultados corretos são, respectivamente, 24 e 10, mas em vez disso obtemos:

In [7]:
df_dx

24.000039999805264

In [8]:
df_dy

10.000000000331966

Note que para calcular ambas as derivadas parciais, temos que chamar `f()` pelo menos três vezes (chamamos quatro vezes no código anterior, mas poderia ser otimizado).

Podemos escrever uma função mais flexível que suporte qualquer número de variáveis e calcule o vetor gradiente contendo todas as derivadas parciais da função em relação a cada variável:

In [9]:
def gradients(func, vars_list, eps=0.0001):
    """
    Calcula as derivadas parciais aproximadas de uma função usando diferenças finitas.

    Args:
        func (function): A função para a qual calcular as derivadas parciais.
        vars_list (list): Uma lista de valores das variáveis de entrada.
        eps (float, opcional): O pequeno incremento usado para aproximar as derivadas (padrão: 0.0001).

    Returns:
        list: Uma lista contendo as derivadas parciais aproximadas para cada variável em vars_list.
    """
    partial_derivatives = []
    base_func_eval = func(*vars_list)
    for idx in range(len(vars_list)):
        tweaked_vars = vars_list[:]
        tweaked_vars[idx] += eps
        tweaked_func_eval = func(*tweaked_vars)
        derivative = (tweaked_func_eval - base_func_eval) / eps
        partial_derivatives.append(derivative)
    return partial_derivatives

In [10]:
def df(x, y):
    return gradients(f, [x, y])

In [14]:
df(3, 4)

(24, 10)

Funcionou!

A boa notícia é que é bastante fácil calcular as Hessianos.

Primeiro, vamos criar funções que calculam as derivadas parciais de primeira ordem (também conhecidas como Jacobianos):

In [ ]:
def dfdx(x, y):
    """
    Calcula a derivada parcial aproximada de f em relação a x usando diferenças finitas.

    Args:
        x (float): O valor de x.
        y (float): O valor de y.

    Returns:
        float: A derivada parcial ∂f/∂x aproximada em (x, y).
    """
    return gradients(f, [x, y])[0]

def dfdy(x, y):
    """
    Calcula a derivada parcial aproximada de f em relação a y usando diferenças finitas.

    Args:
        x (float): O valor de x.
        y (float): O valor de y.

    Returns:
        float: A derivada parcial ∂f/∂y aproximada em (x, y).
    """
    return gradients(f, [x, y])[1]

# Calcula as derivadas parciais em x=3 e y=4
dfdx(3., 4.), dfdy(3., 4.)

(24.000400000048216, 10.000000000047748)

Agora podemos simplesmente aplicar a função `gradients()` a essas funções:

In [ ]:
def d2f(x, y):
    """
    Calcula as derivadas parciais de segunda ordem (Hessiana) da função f(x, y) = x²y + y + 2
    usando diferenças finitas aplicadas às funções de derivadas parciais de primeira ordem.

    Args:
        x (float): O valor de x.
        y (float): O valor de y.

    Returns:
        list: Uma lista contendo duas listas que representam a matriz Hessiana aproximada:
              [[∂²f/∂x∂x, ∂²f/∂x∂y], [∂²f/∂y∂x, ∂²f/∂y∂y]].
    """
    return [gradients(dfdx, [x, y]), gradients(dfdy, [x, y])]

In [11]:
d2f(3, 4)

([8, 6], [6, 0])

Portanto, tudo funciona bem, mas o resultado é aproximado, e calcular os gradientes de uma função em relação a $n$ variáveis requer chamar essa função $n$ vezes. Em redes neurais profundas, pode haver milhões ou até bilhões de parâmetros para ajustar usando o gradiente descendente (o que requer o cálculo dos gradientes da função de perda em relação a cada um desses parâmetros), portanto, essa abordagem seria muito lenta.

No entanto, esse método é tão simples de implementar que é uma ótima ferramenta para verificar se os outros métodos estão implementados corretamente. Por exemplo, se ele discordar da sua função derivada manualmente, então sua função provavelmente contém um erro.

Até agora, consideramos duas maneiras de calcular gradientes: usando diferenciação manual e usando aproximação por diferenças finitas. Infelizmente, ambas são fatalmente falhas para treinar uma rede neural em larga escala. Então, vamos nos voltar para o *autodiff*, começando com o modo direto (*forward mode*).

## Autodiff (Diferenciação Automática)

### Grafo de Computação ( um exemplo simples)

Em vez dessa abordagem numérica, vamos implementar agora algumas técnicas simbólicas de autodiferenciação.

Para isso, precisaremos definir classes para representar constantes, variáveis e operações em um grafo de computação

In [ ]:
class Const(object):
    """
    Representa uma constante no grafo de computação.
    """
    def __init__(self, value):
        self.value = value

    def evaluate(self):
        """
        Retorna o valor da constante.
        """
        return self.value

    def __str__(self):
        return str(self.value)

class Var(object):
    """
    Representa uma variável no grafo de computação.
    """
    def __init__(self, name, init_value=0):
        self.value = init_value
        self.name = name

    def evaluate(self):
        """
        Retorna o valor atual da variável.
        """
        return self.value

    def __str__(self):
        return self.name

class OperadorBinario(object):
    """
    Classe base para operadores binários no grafo de computação.
    """
    def __init__(self, a, b):
        self.a = a
        self.b = b

class Add(OperadorBinario):
    """
    Representa a soma de dois nós no grafo de computação.
    """
    def evaluate(self):
        """
        Retorna o resultado da soma dos dois operandos.
        """
        return self.a.evaluate() + self.b.evaluate()

    def __str__(self):
        return "{} + {}".format(self.a, self.b)

class Mul(OperadorBinario):
    """
    Representa a multiplicação de dois nós no grafo de computação.
    """
    def evaluate(self):
        """
        Retorna o resultado da multiplicação dos dois operandos.
        """
        return self.a.evaluate() * self.b.evaluate()

    def __str__(self):
        return "({}) * ({})".format(self.a, self.b)

Ótimo, agora podemos vizualizar um exemplo de grafo de computação para representar a função $f$:

In [27]:
import base64
from IPython.display import Image, display

# Define o código do gráfico em formato Mermaid para representar o grafo de computação da função f(x, y) = x²y + y + 2
graph = """
graph TD
    X[Entrada: x] --> Mult1(("Mul"))
    X --> Mult1
    Mult1 -- "x²" --> Mult2(("Mul"))
    Y[Entrada: y] --> Mult2
    Mult2 -- "x²y" --> Add1(("Add$"))
    Y --> Add1
    Add1 -- "x²y + y" --> Add2(("Add$"))
    C2[Constante: 2] --> Add2
    Add2 -- "x²y + y + 2" --> F[Saída: f]
"""

# Codifica o gráfico em base64 para gerar uma URL válida para o serviço Mermaid
graphbytes = graph.encode("utf-8")
base64_bytes = base64.b64encode(graphbytes)
base64_string = base64_bytes.decode("ascii")
url = "https://mermaid.ink/img/" + base64_string

# Exibe a imagem do gráfico gerado
display(Image(url=url))

In [16]:
x = Var("x")
y = Var("y")
f = Add(Mul(Mul(x, x), y), Add(y, Const(2))) # f(x,y) = x²y + y + 2

E podemos executar este gráfico para calcular $f$ em qualquer ponto, por exemplo, $f(3, 4)$.

In [17]:
x.value = 3
y.value = 4
f.evaluate()

42

Perfeito, encontramos a resposta definitiva!

### Calculando Gradientes com Grafos de Computação

Os métodos de autodiff que apresentaremos abaixo são todos baseados na "regra da cadeia".

Suponha que temos duas funções $u$ e $v$, e as aplicamos sequencialmente a uma entrada $x$, obtendo o resultado $z$. Assim, temos $z = v(u(x))$, o que podemos reescrever como $z = v(s)$ e $s = u(x)$. Agora podemos aplicar a regra da cadeia para obter a derivada parcial da saída $z$ em relação à entrada $x$:

$ \dfrac{\partial z}{\partial x} = \dfrac{\partial s}{\partial x} \cdot \dfrac{\partial z}{\partial s}$

Agora, se $z$ for a saída de uma sequência de funções que possuem saídas intermediárias $s_1, s_2, ..., s_n$, a regra da cadeia ainda se aplica:

$ \dfrac{\partial z}{\partial x} = \dfrac{\partial s_1}{\partial x} \cdot \dfrac{\partial s_2}{\partial s_1} \cdot \dfrac{\partial s_3}{\partial s_2} \cdot \dots \cdot \dfrac{\partial s_{n-1}}{\partial s_{n-2}} \cdot \dfrac{\partial s_n}{\partial s_{n-1}} \cdot \dfrac{\partial z}{\partial s_n}$

Na autodiff de modo direto (*forward mode*), o algoritmo calcula esses termos "para frente" (isto é, na mesma ordem que os cálculos necessários para computar a saída $z$), ou seja, da esquerda para a direita:

- primeiro $\dfrac{\partial s_1}{\partial x}$, depois $\dfrac{\partial s_2}{\partial s_1}$, e assim por diante.

Na autodiff de modo reverso (*reverse mode*), o algoritmo calcula esses termos "para trás", da direita para a esquerda:
- primeiro $\dfrac{\partial z}{\partial s_n}$, depois $\dfrac{\partial s_n}{\partial s_{n-1}}$, e assim por diante.

Por exemplo, suponha que você queira calcular a derivada da função $z(x)=\sin(x^2)$ em $x=3$, usando a autodiff de modo direto.

- O algoritmo calcularia primeiro a derivada parcial $\dfrac{\partial s_1}{\partial x}=\dfrac{\partial x^2}{\partial x}=2x$, que é 6 em $x=3$.

- Em seguida, ele calcularia $\dfrac{\partial z}{\partial x}=\dfrac{\partial s_1}{\partial x}\cdot\dfrac{\partial z}{\partial s_1}= 6 \cdot \dfrac{\partial \sin(s_1)}{\partial s_1}=6 \cdot \cos(s_1)=6 \cdot \cos(3^2)\approx-5,46$.

Vamos verificar o resultado com a função `gradients()` que definimos anteriormente:

In [29]:
from math import sin

def z(x):
    return sin(x**2)

gradients(z, [3])

[-5.46761419430053]

Muito bom!.

Agora vamos fazer a mesma coisa usando a autodiff de modo reverso.

- Desta vez, o algoritmo começaria do lado direito, então cacularíamos

$\dfrac{\partial z}{\partial s_1} = \dfrac{\partial \sin(s_1)}{\partial s_1}=\cos(s_1)=\cos(3^2)\approx -0,91$.

- Em seguida,

$\dfrac{\partial z}{\partial x}=\dfrac{\partial s_1}{\partial x}\cdot\dfrac{\partial z}{\partial s_1} \approx \dfrac{\partial s_1}{\partial x} \cdot -0,91 = \dfrac{\partial x^2}{\partial x} \cdot -0,91=2x \cdot -0,91 = 6\cdot-0,91=-5,46$.

É claro que ambas as abordagens fornecem o mesmo resultado (exceto por erros de arredondamento) e, com uma única entrada e saída, envolvem o mesmo número de cálculos.
- Mas quando há várias entradas ou saídas, elas podem ter um desempenho muito diferente.

De fato, se houver muitas entradas, os termos mais à direita serão necessários para calcular as derivadas parciais em relação a cada entrada, portanto, é uma boa ideia calcular esses termos da direita primeiro.
- Isso significa usar a autodiff de modo reverso.

Dessa forma, os termos mais à direita podem ser calculados apenas uma vez e usados para calcular todas as derivadas parciais.

Por outro lado, se houver muitas saídas, o modo direto é geralmente preferível, porque os termos mais à esquerda podem ser calculados apenas uma vez para computar as derivadas parciais das diferentes saídas.

Em Deep Learning, normalmente existem milhares de parâmetros de modelo, o que significa que há muitas entradas, mas poucas saídas.

Na verdade, geralmente há apenas uma saída durante o treinamento: a perda. É por isso que a autodiff de modo reverso é usada no PyTorch e em todas as principais bibliotecas de Deep Learning.

Há uma complexidade adicional na autodiff de modo reverso: o valor de $s_i$ é geralmente necessário ao calcular $\dfrac{\partial s_{i+1}}{\partial s_i}$, e calcular $s_i$ requer primeiro calcular $s_{i-1}$, que requer calcular $s_{i-2}$, e assim por diante. Basicamente, é necessária uma primeira passada direta (forward pass) pela rede para calcular $s_1$, $s_2$, $s_3$, $\dots$, $s_{n-1}$ e $s_n$, para que então o algoritmo possa calcular as derivadas parciais da direita para a esquerda.Armazenar todos os valores intermediários $s_i$ na RAM às vezes é um problema, especialmente ao lidar com imagens e ao usar GPUs, que frequentemente têm RAM limitada: para limitar esse problema, pode-se reduzir o número de camadas na rede neural ou configurar o PyTorch para transferir (swap) esses valores da RAM da GPU para a RAM da CPU.Outra abordagem é armazenar em cache apenas valores intermediários alternados, $s_1$, $s_3$, $s_5$, $\dots$, $s_{n-4}$, $s_{n-2}$ e $s_n$. Isso significa que, quando o algoritmo calcula as derivadas parciais, se um valor intermediário $s_i$ estiver faltando, ele precisará recalculá-lo com base no valor intermediário anterior $s_{i-1}$. Isso troca processamento (CPU) por memória (RAM) (se você estiver interessado, confira [este artigo](https://pdfs.semanticscholar.org/f61e/9fd5a4878e1493f7a6b03774a61c17b7e9a4.pdf)).

### Autodiff de Modo Direto (Forward-Mode)

A [Figura A-1](#figure-a-1) mostra como o *autodiff* de modo direto funciona em uma função ainda mais simples, $g(x, y) = 5 + xy$. O grafo para essa função é representado à esquerda. Após o *autodiff* de modo direto, obtemos o grafo à direita, que representa a derivada parcial $\partial g/\partial x = 0 + (0 \times x + y \times 1) = y$ (poderíamos obter similarmente a derivada parcial em relação a $y$).

O algoritmo percorrerá o grafo de computação das entradas para as saídas (daí o nome "modo direto"). Ele começa obtendo as derivadas parciais dos nós folha. O nó constante (5) retorna a constante 0, já que a derivada de uma constante é sempre 0. A variável $x$ retorna a constante 1, já que $\partial x/\partial x = 1$, e a variável $y$ retorna a constante 0, já que $\partial y/\partial x = 0$ (se estivéssemos procurando a derivada parcial em relação a $y$, seria o inverso).

Agora temos tudo o que precisamos para subir no grafo até o nó de multiplicação na função $g$. O cálculo nos diz que a derivada do produto de duas funções $u$ e $v$ é $\partial(u \times v)/\partial x = \partial v/\partial x \times u + v \times \partial u/\partial x$. Podemos, portanto, construir uma grande parte do grafo à direita, representando $0 \times x + y \times 1$.

Finalmente, podemos subir até o nó de adição na função $g$. Como mencionado, a derivada de uma soma de funções é a soma das derivadas dessas funções, então precisamos apenas criar um nó de adição e conectá-lo às partes do grafo que já calculamos. Obtemos a derivada parcial correta: $\partial g/\partial x = 0 + (0 \times x + y \times 1)$.




<a id="figure-a-1"></a>


![](https://github.com/fabiobento/dnn-course-2026-1/raw/main/images/figure-a-1.png)

**Figura A-1. Autodiff de modo direto**([Fonte](https://ageron.github.io/))

No entanto, essa equação pode ser simplificada (e muito). Aplicando alguns passos de poda ao grafo de computação para se livrar de todas as operações desnecessárias, obtemos um grafo muito menor com apenas um nó: $\partial g/\partial x = y$. Neste caso, a simplificação é bastante fácil, mas para uma função mais complexa, o *autodiff* de modo direto pode produzir um grafo enorme que pode ser difícil de simplificar e levar a um desempenho subótimo.

Note que começamos com um grafo de computação, e o *autodiff* de modo direto produziu outro grafo de computação. Isso é chamado de *diferenciação simbólica*, e tem duas características interessantes. Primeiro, uma vez produzido o grafo de computação da derivada, podemos usá-lo quantas vezes quisermos para calcular as derivadas da função dada para qualquer valor de $x$ e $y$. Segundo, podemos rodar o *autodiff* de modo direto novamente no grafo resultante para obter derivadas de segunda ordem, se precisarmos (ou seja, derivadas de derivadas). Poderíamos até calcular derivadas de terceira ordem, e assim por diante.

Mas também é possível rodar o *autodiff* de modo direto sem construir um grafo (ou seja, numericamente, não simbolicamente) apenas computando resultados intermediários em tempo real (*on the fly*). Uma maneira de fazer isso é usar *números duais*, que são números estranhos, mas fascinantes, da forma $a + b\epsilon$, onde $a$ e $b$ são números reais, e $\epsilon$ é um número infinitesimal tal que $\epsilon^2 = 0$ (mas $\epsilon \neq 0$). Você pode pensar no número dual $42 + 24\epsilon$ como algo semelhante a $42.0000\dots000024$ com um número infinito de zeros (mas é claro que isso é apenas para lhe dar uma intuição do que são os números duais).

Os números duais podem ser somados, multiplicados, divididos, etc. Por exemplo:
*   $\lambda(a + b\epsilon) = \lambda a + \lambda b\epsilon$
*   $(a + b\epsilon) + (c + d\epsilon) = (a + c) + (b + d)\epsilon$
*   $(a + b\epsilon) \times (c + d\epsilon) = ac + (ad + bc)\epsilon + bd\epsilon^2 = ac + (ad + bc)\epsilon$

Mais importante ainda, pode-se mostrar que $h(a + b\epsilon) = h(a) + b \times h'(a)\epsilon$, então calcular $h(a + \epsilon)$ dá $h(a) + h'(a)\epsilon$. Isso significa que podemos calcular tanto $h(a)$ quanto $h'(a)$ de uma só vez, simplesmente calculando $h(a + \epsilon)$.

A Figura A-2 mostra como calcular a derivada parcial de $f(x, y) = x^2y + y + 2$ em relação a $x$ em $x = 3$ e $y = 4$. Apenas aplicamos a função $f$ aos números duais $\hat{x} = 3 + \epsilon$ e $\hat{y} = 4$.

**Figura A-2. Autodiff de modo direto usando números duais**

O resultado é $42 + 24\epsilon$, o que significa que $f(3, 4) = 42$ e a derivada parcial $\partial f/\partial x(3, 4) = 24$.

Para calcular $\partial f/\partial y (3, 4)$, teríamos que passar pelo grafo novamente, mas desta vez com $x = 3$ e $y = 4 + \epsilon$.

Assim, o *autodiff* de modo direto é muito mais preciso que a aproximação por diferenças finitas, mas sofre da mesma falha principal, pelo menos quando há muitas entradas e poucas saídas (como é o caso ao lidar com redes neurais): se houvesse 1.000 parâmetros, ele exigiria 1.000 passagens pelo grafo para calcular todas as derivadas parciais. É aqui que o *autodiff* de modo reverso brilha: ele pode calcular todas elas em apenas duas passagens pelo grafo. Vamos ver como.


Now that we know the theory, let's implement forward mode autodiff.

In [19]:
Const.gradient = lambda self, var: Const(0)
Var.gradient = lambda self, var: Const(1) if self is var else Const(0)
Add.gradient = lambda self, var: Add(self.a.gradient(var), self.b.gradient(var))
Mul.gradient = lambda self, var: Add(Mul(self.a, self.b.gradient(var)), Mul(self.a.gradient(var), self.b))

x = Var(name="x", init_value=3.)
y = Var(name="y", init_value=4.)
f = Add(Mul(Mul(x, x), y), Add(y, Const(2))) # f(x,y) = x²y + y + 2

dfdx = f.gradient(x)  # 2xy
dfdy = f.gradient(y)  # x² + 1

In [20]:
dfdx.evaluate(), dfdy.evaluate()

(24.0, 10.0)

Since the output of the `gradient()` method is fully symbolic, we are not limited to the first order derivatives, we can also compute second order derivatives, and so on:

In [21]:
d2fdxdx = dfdx.gradient(x) # 2y
d2fdxdy = dfdx.gradient(y) # 2x
d2fdydx = dfdy.gradient(x) # 2x
d2fdydy = dfdy.gradient(y) # 0

In [22]:
[[d2fdxdx.evaluate(), d2fdxdy.evaluate()],
 [d2fdydx.evaluate(), d2fdydy.evaluate()]]

[[8.0, 6.0], [6.0, 0.0]]

Note that the result is now exact, not an approximation (up to the limit of the machine's float precision, of course).

#### Forward mode autodiff using dual numbers

A nice way to apply forward mode autodiff is to use [dual numbers](https://en.wikipedia.org/wiki/Dual_number). In short, a dual number $z$ has the form $z = a + b\epsilon$, where $a$ and $b$ are real numbers, and $\epsilon$ is an infinitesimal number, positive but smaller than all real numbers, and such that $\epsilon^2=0$.
It can be shown that $f(x + \epsilon) = f(x) + \dfrac{\partial f}{\partial x}\epsilon$, so simply by computing $f(x + \epsilon)$ we get both the value of $f(x)$ and the partial derivative of $f$ with regards to $x$.

Dual numbers have their own arithmetic rules, which are generally quite natural. For example:

**Addition**

$(a_1 + b_1\epsilon) + (a_2 + b_2\epsilon) = (a_1 + a_2) + (b_1 + b_2)\epsilon$

**Subtraction**

$(a_1 + b_1\epsilon) - (a_2 + b_2\epsilon) = (a_1 - a_2) + (b_1 - b_2)\epsilon$

**Multiplication**

$(a_1 + b_1\epsilon) \times (a_2 + b_2\epsilon) = (a_1 a_2) + (a_1 b_2 + a_2 b_1)\epsilon + b_1 b_2\epsilon^2 = (a_1 a_2) + (a_1b_2 + a_2b_1)\epsilon$

**Division**

$\dfrac{a_1 + b_1\epsilon}{a_2 + b_2\epsilon} = \dfrac{a_1 + b_1\epsilon}{a_2 + b_2\epsilon} \cdot \dfrac{a_2 - b_2\epsilon}{a_2 - b_2\epsilon} = \dfrac{a_1 a_2 + (b_1 a_2 - a_1 b_2)\epsilon - b_1 b_2\epsilon^2}{{a_2}^2 + (a_2 b_2 - a_2 b_2)\epsilon - {b_2}^2\epsilon} = \dfrac{a_1}{a_2} + \dfrac{a_1 b_2 - b_1 a_2}{{a_2}^2}\epsilon$

**Power**

$(a + b\epsilon)^n = a^n + (n a^{n-1}b)\epsilon$

etc.

Let's create a class to represent dual numbers, and implement a few operations (addition and multiplication). You can try adding some more if you want.

In [23]:
class DualNumber(object):
    def __init__(self, value=0.0, eps=0.0):
        self.value = value
        self.eps = eps
    def __add__(self, b):
        return DualNumber(self.value + self.to_dual(b).value,
                          self.eps + self.to_dual(b).eps)
    def __radd__(self, a):
        return self.to_dual(a).__add__(self)
    def __mul__(self, b):
        return DualNumber(self.value * self.to_dual(b).value,
                          self.eps * self.to_dual(b).value + self.value * self.to_dual(b).eps)
    def __rmul__(self, a):
        return self.to_dual(a).__mul__(self)
    def __str__(self):
        if self.eps:
            return "{:.1f} + {:.1f}ε".format(self.value, self.eps)
        else:
            return "{:.1f}".format(self.value)
    def __repr__(self):
        return str(self)
    @classmethod
    def to_dual(cls, n):
        if hasattr(n, "value"):
            return n
        else:
            return cls(n)

$3 + (3 + 4 \epsilon) = 6 + 4\epsilon$

In [24]:
3 + DualNumber(3, 4)

6.0 + 4.0ε

$(3 + 4ε)\times(5 + 7ε)$ = $3 \times 5 + 3 \times 7ε + 4ε \times 5 + 4ε \times 7ε$ = $15 + 21ε + 20ε + 28ε^2$ = $15 + 41ε + 28 \times 0$ = $15 + 41ε$

In [25]:
DualNumber(3, 4) * DualNumber(5, 7)

15.0 + 41.0ε

Now let's see if the dual numbers work with our toy computation framework:

In [26]:
x.value = DualNumber(3.0)
y.value = DualNumber(4.0)

f.evaluate()

42.0

Yep, sure works. Now let's use this to compute the partial derivatives of $f$ with regards to $x$ and $y$ at x=3 and y=4:

In [27]:
x.value = DualNumber(3.0, 1.0)  # 3 + ε
y.value = DualNumber(4.0)       # 4

dfdx = f.evaluate().eps

x.value = DualNumber(3.0)       # 3
y.value = DualNumber(4.0, 1.0)  # 4 + ε

dfdy = f.evaluate().eps

In [28]:
dfdx

24.0

In [29]:
dfdy

10.0

Great! However, in this implementation we are limited to first order derivatives.
Now let's look at reverse mode.

### Autodiff de Modo Reverso (Reverse-Mode)

O *autodiff* de modo reverso é a solução implementada pelo PyTorch. Ele primeiro percorre o grafo na direção direta (ou seja, das entradas para a saída) para calcular o valor de cada nó. Em seguida, faz uma segunda passagem, desta vez na direção reversa (ou seja, da saída para as entradas) para calcular todas as derivadas parciais. O nome "modo reverso" vem dessa segunda passagem pelo grafo, onde os gradientes fluem na direção reversa. A Figura A-3 representa a segunda passagem. Durante a primeira passagem, todos os valores dos nós foram calculados, começando de $x = 3$ e $y = 4$. Você pode ver esses valores no canto inferior direito de cada nó (por exemplo, $x \times x = 9$). Os nós são rotulados de $n_1$ a $n_7$ para clareza. O nó de saída é $n_7$: $f(3, 4) = n_7 = 42$.

**Figura A-3. Autodiff de modo reverso**

A ideia é descer gradualmente pelo grafo, calculando a derivada parcial de $f(x, y)$ em relação a cada nó consecutivo, até alcançarmos os nós variáveis. Para isso, o *autodiff* de modo reverso baseia-se fortemente na *regra da cadeia*, mostrada na Equação A-4.

**Equação A-4. Regra da cadeia**

$$
\frac{\partial f}{\partial x} = \frac{\partial f}{\partial n_i} \times \frac{\partial n_i}{\partial x}
$$

Como $n_7$ é o nó de saída, $f = n_7$, então $\partial f / \partial n_7 = 1$.

Vamos continuar descendo o grafo até $n_5$: quanto $f$ varia quando $n_5$ varia? A resposta é $\partial f / \partial n_5 = \partial f / \partial n_7 \times \partial n_7 / \partial n_5$. Já sabemos que $\partial f / \partial n_7 = 1$, então tudo o que precisamos é $\partial n_7 / \partial n_5$. Como $n_7$ simplesmente realiza a soma $n_5 + n_6$, descobrimos que $\partial n_7 / \partial n_5 = 1$, então $\partial f / \partial n_5 = 1 \times 1 = 1$.

Agora podemos prosseguir para o nó $n_4$: quanto $f$ varia quando $n_4$ varia? A resposta é $\partial f / \partial n_4 = \partial f / \partial n_5 \times \partial n_5 / \partial n_4$. Como $n_5 = n_4 \times n_2$, descobrimos que $\partial n_5 / \partial n_4 = n_2$, então $\partial f / \partial n_4 = 1 \times n_2 = 4$.

O processo continua até alcançarmos a base do grafo. Nesse ponto, teremos calculado todas as derivadas parciais de $f(x, y)$ no ponto $x = 3$ e $y = 4$. Neste exemplo, encontramos $\partial f / \partial x = 24$ e $\partial f / \partial y = 10$. Parece correto!

O *autodiff* de modo reverso é uma técnica muito poderosa e precisa, especialmente quando há muitas entradas e poucas saídas, pois requer apenas uma passagem direta mais uma passagem reversa por saída para calcular todas as derivadas parciais para todas as saídas em relação a todas as entradas. Ao treinar redes neurais, geralmente queremos minimizar a perda (*loss*), então há uma única saída (a perda) e, portanto, apenas duas passagens pelo grafo são necessárias para calcular os gradientes.

O PyTorch constrói um novo grafo dinamicamente (*on the fly*) durante cada passagem direta. Sempre que você executa uma operação em um tensor com `requires_grad=True`, o PyTorch calcula o tensor resultante e define seu atributo `grad_fn` para um objeto específico da operação que permite ao PyTorch propagar os gradientes para trás através dessa operação. Como o grafo é construído em tempo de execução, seu código pode ser altamente dinâmico, contendo loops e condicionais, e tudo ainda funcionará bem.

O *autodiff* de modo reverso também pode lidar com funções que não são inteiramente diferenciáveis, desde que você peça para calcular as derivadas parciais em pontos que são diferenciáveis.

Let's rewrite our toy framework to add reverse mode autodiff:

In [30]:
class Const(object):
    def __init__(self, value):
        self.value = value
    def evaluate(self):
        return self.value
    def backpropagate(self, gradient):
        pass
    def __str__(self):
        return str(self.value)

class Var(object):
    def __init__(self, name, init_value=0):
        self.value = init_value
        self.name = name
        self.gradient = 0
    def evaluate(self):
        return self.value
    def backpropagate(self, gradient):
        self.gradient += gradient
    def __str__(self):
        return self.name

class BinaryOperator(object):
    def __init__(self, a, b):
        self.a = a
        self.b = b

class Add(BinaryOperator):
    def evaluate(self):
        self.value = self.a.evaluate() + self.b.evaluate()
        return self.value
    def backpropagate(self, gradient):
        self.a.backpropagate(gradient)
        self.b.backpropagate(gradient)
    def __str__(self):
        return "{} + {}".format(self.a, self.b)

class Mul(BinaryOperator):
    def evaluate(self):
        self.value = self.a.evaluate() * self.b.evaluate()
        return self.value
    def backpropagate(self, gradient):
        self.a.backpropagate(gradient * self.b.value)
        self.b.backpropagate(gradient * self.a.value)
    def __str__(self):
        return "({}) * ({})".format(self.a, self.b)

In [31]:
x = Var("x", init_value=3)
y = Var("y", init_value=4)
f = Add(Mul(Mul(x, x), y), Add(y, Const(2))) # f(x,y) = x²y + y + 2

result = f.evaluate()
f.backpropagate(1.0)

In [32]:
print(f)

((x) * (x)) * (y) + y + 2


In [33]:
result

42

In [34]:
x.gradient

24.0

In [35]:
y.gradient

10.0

Again, in this implementation the outputs are just numbers, not symbolic expressions, so we are limited to first order derivatives. However, we could have made the `backpropagate()` methods return symbolic expressions rather than values (e.g., return `Add(2,3)` rather than 5). This would make it possible to compute second order gradients (and beyond).

#### Reverse mode autodiff using PyTorch

PyTorch includes a system called `autograd` which implements reverse-mode autodiff:

In [36]:
import torch

In [37]:
x = torch.tensor(3., requires_grad=True)
y = torch.tensor(4., requires_grad=True)

f = x*x*y + y + 2

f.backward() # run backpropagation
jacobians = [x.grad, y.grad]
jacobians

[tensor(24.), tensor(10.)]

Easy!

Now let's try computing second-order derivatives:

In [38]:
x = torch.tensor(3., requires_grad=True)
y = torch.tensor(4., requires_grad=True)

f = x*x*y + y + 2

dfdx = torch.autograd.grad(f, x, create_graph=True)[0]
dfdy = torch.autograd.grad(f, y, create_graph=True)[0]

d2fdxdx = torch.autograd.grad(dfdx, x, retain_graph=True)[0]
d2fdxdy = torch.autograd.grad(dfdx, y)[0]
d2fdydx = torch.autograd.grad(dfdy, x, retain_graph=True)[0]
d2fdydy = torch.autograd.grad(dfdy, y, allow_unused=True)[0]

hessians = [[d2fdxdx, d2fdxdy], [d2fdydx, d2fdydy]]
hessians

[[tensor(8.), tensor(6.)], [tensor(6.), None]]

It works too, great! However, this was not as easy, let's go through this code:
* we first create `x`, `y`, and `f`, just like earlier.
* then we use the `torch.autograd.grad()` function to compute the first-order gradients `dfdx` and `dfdy`. We use `grad()` instead of `backward()` because it gives us more control, which is needed here. Note that we set `create_graph=True` here because we need graphs (i.e., symbolic representations) instead of numerical values (which is the default), so that we can then compute the second-order derivatives based on these graphs.
* We then compute each of the four second-order derivatives using the `grad()` function.
  * When we compute `d2fdxdx`, we must specify `retain_graph=True`, or else by default PyTorch will try to save RAM by deleting anything it doesn't need anymore in the computation graph for `dfdx`, as it computes the gradients `d2fdxdx`. As a result, we wouldn't be able to compute `d2fdxdy`, since it also needs the graph for `dfdx`.
  * Similarly, we need to set `retain_graph=True` when computing `d2fdydx`, or else we wouldn't be able to compute `d2fdydy`.
  * Lastly, we must set `allow_unused=True` when computing `d2fdydy` because the first-order partial derivative $\dfrac{\partial f}{\partial y} = x^2 + 1$, which doesn't use `y` at all. That's why we get `None` for `d2fdydy` (without `allow_unused=True`, we would get an exception instead).

And that's all folks! Hope you enjoyed this notebook.